In [2]:
import numpy as np
import glob
import matplotlib.pyplot as plt
import cv2
from skimage import measure, segmentation
from vis_utils import load_volume, VolumeVisualizer
from scipy.ndimage import zoom
from skimage.morphology import skeletonize, skeletonize_3d
from skimage import filters, morphology

from scipy.ndimage.filters import convolve, correlate
from scipy import signal

In [3]:
source_dir = './data/'
files = list(sorted(glob.glob(source_dir + '/*/*.raw')))
list(enumerate(files))

[(0, './data/P01/P01_60um_1612x623x1108.raw'),
 (1, './data/P02/P02_60um_1387x778x1149.raw'),
 (2, './data/P03/P03_60um_1473x1163x1148.raw'),
 (3, './data/P04/P04_60um_1273x466x1045.raw'),
 (4, './data/P05/P05_60um_1454x817x1102.raw'),
 (5, './data/P06/P06_60um_1425x564x1028.raw'),
 (6, './data/P07/P7_60um_1216x692x926.raw'),
 (7, './data/P08/P08_60um_1728x927x1149.raw'),
 (8, './data/P09/P09_60um_1359x456x1040.raw'),
 (9, './data/P10/P10_60um_1339x537x1035.raw'),
 (10, './data/P11/P11_60um_1735x595x1150.raw'),
 (11, './data/P12/P12_60um_1333x443x864.raw'),
 (12, './data/P13/P13_60um_1132x488x877.raw'),
 (13, './data/P14/P14_60um_1927x746x1124.raw'),
 (14, './data/P15/P15_60um_1318x640x1059.raw'),
 (15, './data/P16/P16_60um_1558x687x1084.raw'),
 (16, './data/P17/P17_60um_1573x555x968.raw'),
 (17, './data/P320/320_60um_1739x553x960.raw'),
 (18, './data/P333/333_60um_1762x989x1095.raw'),
 (19, './data/P73/73_60um_1729x854x1143.raw')]

In [4]:
%%time
volume = load_volume(files[11], scale=0.5)
print(volume.shape, volume.size)

(432, 222, 666) 63872064
CPU times: user 1.08 s, sys: 799 ms, total: 1.88 s
Wall time: 2.19 s


In [4]:
visualizer = VolumeVisualizer(volume, binary=False).visualize()

## simple threshold segmentation

In [5]:
threshold = 70
mask = volume > threshold

VolumeVisualizer(mask).visualize()

In [6]:
def get_main_regions(binary_mask, min_size=10_000, connectivity=3):
    labeled = measure.label(binary_mask, connectivity=connectivity)
    region_props = measure.regionprops(labeled)
    
    main_regions_masks = []
    
    for props in region_props:
        if props.area >= min_size:
            main_regions_masks.append((props.filled_image, props.bbox))
            
    return main_regions_masks

def merge_masks(masks, img_shape):
    result_mask = np.zeros(img_shape, dtype=np.uint8)
    for mask, bbox in masks:
        min1, min2, min3, max1, max2, max3 = bbox
        result_mask[min1:max1, min2:max2, min3:max3] += mask.astype(np.uint8)
        
    return result_mask

In [7]:
%%time
main_regions_masks = get_main_regions(mask, min_size=5_000, connectivity=1)
len(main_regions_masks)

CPU times: user 10.5 s, sys: 339 ms, total: 10.9 s
Wall time: 10.9 s


1

In [8]:
VolumeVisualizer(
    main_regions_masks[0][0], binary=False
).visualize(scale=1, primary_color=(255,255,255))

In [9]:
merged_mask = merge_masks(main_regions_masks, mask.shape)
merged_mask[50:180, 50:160, 70:200] = 0.5

visualizer = VolumeVisualizer(
    merged_mask * 100
).visualize(scale=1, primary_color=(255,255,255))

In [10]:
main_regions_masks[0][0].max()

True

In [11]:
main_mask = main_regions_masks[0][0]

## generating spherical kernel

In [12]:
def spherical_kernel(outer_radius, thickness=1, filled=False):
   
    inner_radius = outer_radius - thickness
    
    outer_sphere = morphology.ball(radius=outer_radius)
    if filled:
        return outer_sphere
    
    inner_sphere = morphology.ball(radius=inner_radius)
    
    begin = outer_radius - inner_radius
    end = begin + inner_sphere.shape[0]
    
    outer_sphere[begin:end, begin:end, begin:end] -= inner_sphere
    return outer_sphere

## convolving with 3d kernel

In [13]:
def convolve_with_ball(mask, ball_radius, dtype=np.uint16):
    kernel = spherical_kernel(ball_radius, filled=True)
    return signal.convolve(mask.astype(dtype), kernel.astype(dtype), mode='valid')

In [52]:
%%time

convolved = convolve_with_ball(mask, ball_radius=8)
print(convolved.shape, convolved.max())

(416, 206, 650) 1868
CPU times: user 8.15 s, sys: 4.71 s, total: 12.9 s
Wall time: 14.7 s


In [53]:
VolumeVisualizer((convolved / convolved.max() * 255).astype(np.uint8), binary=False
                ).visualize(primary_color=(255,255,255))

## extracting main arterial regions from convolved image

In [37]:
def get_arterial_regions(conv_img, lower_hist_fraction, upper_hist_fraction):
    upper_hist_value = upper_hist_fraction * conv_img.max()
    lower_hist_value = lower_hist_fraction * conv_img.max()
    return filters.apply_hysteresis_threshold(conv_img, lower_hist_value, upper_hist_value)

In [49]:
%%time
arterial_regions = get_arterial_regions(convolved, 0.2, 0.9)
print(measure.label(arterial_regions).max())

1
CPU times: user 2.84 s, sys: 1.83 s, total: 4.67 s
Wall time: 5.67 s


In [50]:
VolumeVisualizer(arterial_regions, binary=True).visualize()

## skeletonization of arterial regions

In [51]:
skeletonized = skeletonize_3d(arterial_regions.astype(np.uint8))
VolumeVisualizer(skeletonized, binary=True).visualize()

## reconstructing arterial structure from skeletonized image

In [47]:
def reconstruct_from_skeleton(skeleton, ball_radius):
    
    mask = np.zeros(skeleton.shape, dtype=np.uint8)
    mask = np.pad(mask, ball_radius)
    
    kernel = spherical_kernel(ball_radius, filled=True)
    central_points = np.argwhere(skeleton == 1)
    
    for central_point in central_points:
        start_corner = tuple(central_point)
        end_corner = tuple(central_point + 2*ball_radius + 1)
        
        start1, start2, start3 = start_corner
        end1, end2, end3 = end_corner
        
        mask_slice = mask[start1:end1, start2:end2, start3:end3]
        mask_slice[:] = np.logical_or(mask_slice, kernel)
        
    return mask[ball_radius:-ball_radius, ball_radius:-ball_radius, ball_radius:-ball_radius]

In [48]:
reconstructed = reconstruct_from_skeleton(skeletonized, 10)
VolumeVisualizer(reconstructed, binary=True).visualize()

In [43]:
skxd = skeletonize_3d(reconstructed.astype(np.uint8))
VolumeVisualizer(skxd, binary=True).visualize()